In [2]:
import os
import json
import pandas as pd
import psycopg2
import streamlit as st
import plotly.express as px


In [3]:
pip install nbformat

  Using cached fastjsonschema-2.19.1-py3-none-any.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/78.4 kB ? eta -:--:--
   ---------------------------------------- 78.4/78.4 kB 4.3 MB/s eta 0:00:00
Using cached fastjsonschema-2.19.1-py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!git clone https://github.com/PhonePe/pulse.git

fatal: destination path 'pulse' already exists and is not an empty directory.


In [5]:
# Aggregated_Transactions

path1="D:/DS/Projects/project 2 - phonepe/pulse/data/aggregated/transaction/country/india/state/" #Pasting the path into a variable as a string
agg_trans_list=os.listdir(path1)    #Convesting the string into a path

column1={"States":[], "Years":[], "Quarters":[], "Transaction_type":[], "Transaction_count":[], "Transaction_amount":[]}

for state1 in agg_trans_list:                # accessing all the states by using for loop
    curr_states1=path1+state1+"/" #creating the location for all the states
    agg_year_list1=os.listdir(curr_states1)   #accessing the year folders
    
    for year1 in agg_year_list1:
        curr_year1=curr_states1+year1+"/"  #Creating the path
        agg_file_list1=os.listdir(curr_year1)     #accessing the JSON files
        
        for file1 in agg_file_list1:
            curr_file=curr_year1+file1
            data1=open(curr_file)

            A=json.load(data1)

            for a in A["data"]["transactionData"]:
                type=a["name"]
                count=a["paymentInstruments"][0]["count"]
                amount=a["paymentInstruments"][0]["amount"]

                column1["States"].append(state1)
                column1["Years"].append(year1)
                column1["Quarters"].append(int(file1.strip(".json")))
                column1["Transaction_type"].append(type)
                column1["Transaction_count"].append(count)
                column1["Transaction_amount"].append(amount)
            

state_mapping = {"andaman-&-nicobar-islands": "Andaman & Nicobar",
"arunachal-pradesh": "Arunachal Pradesh",
"assam": "Assam",
"bihar": "Bihar",
"chandigarh": "Chandigarh",
"chhattisgarh": "Chhattisgarh",
"dadra-&-nagar-haveli-&-daman-&-diu": "Dadra and Nagar Haveli and Daman and Diu",
"Daman and Diu": "Dadra and Nagar Haveli and Daman and Diu",
"goa": "Goa",
"gujarat": "Gujarat",
"haryana": "Haryana",
"himachal-pradesh": "Himachal Pradesh",
"jharkhand": "Jharkhand",
"karnataka": "Karnataka",
"kerala": "Kerala",
"lakshadweep": "Lakshadweep",
"madhya-pradesh": "Madhya Pradesh",
"maharashtra": "Maharashtra",
"manipur": "Manipur",
"meghalaya": "Meghalaya",
"mizoram": "Mizoram",
"nagaland": "Nagaland",
"delhi": "Delhi",
"puducherry": "Puducherry",
"punjab": "Punjab",
"rajasthan": "Rajasthan",
"sikkim": "Sikkim",
"tamil-nadu": "Tamil Nadu",
"telangana": "Telangana",
"tripura": "Tripura",
"uttar-pradesh": "Uttar Pradesh",
"uttarakhand": "Uttarakhand",
"west-bengal": "West Bengal",
"odisha": "Odisha",
"andhra-pradesh": "Andhra Pradesh",
"jammu-&-kashmir": "Jammu & Kashmir",
"ladakh": "Ladakh"}


df1 = pd.DataFrame(column1)
df1["States"] = df1["States"].replace(state_mapping)

        

In [6]:
df1

,States,Years,Quarters,Transaction_type,Transaction_count,Transaction_amount
0,Andaman & Nicobar,2018,1,Recharge & bill payments,4200,1.845307e+06
1,Andaman & Nicobar,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,Andaman & Nicobar,2018,1,Merchant payments,298,4.525072e+05
3,Andaman & Nicobar,2018,1,Financial Services,33,1.060142e+04
4,Andaman & Nicobar,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
4129,West Bengal,2023,3,Merchant payments,296965002,2.004404e+11
4130,West Bengal,2023,3,Peer-to-peer payments,271150280,8.310501e+11
4131,West Bengal,2023,3,Recharge & bill payments,63055684,3.920985e+10
4132,West Bengal,2023,3,Financial Services,389751,4.249856e+08


In [7]:
# Connecting the PostgreSQL

mydb = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="Hannah$2686",
    database="phonepe",
    port="5432")

cursor = mydb.cursor()     # Creating a cursor to interact with the database

drop_query = """drop table if exists Aggregated_Transactions"""  # Drop the 'Aggregated_Transactions' table if it exists
cursor.execute(drop_query)
mydb.commit()

# Create the 'Aggregated_Transactions' table in PostgreSQL if it does not exist.
create_query = """
    create table if not exists Aggregated_Transactions(
    States varchar(100),
    Years int,
    Quarters int,
    Transaction_type text,
    Transaction_count bigint,
    Transaction_amount bigint
    )
    """
cursor.execute(create_query)
mydb.commit()


# Inserting  information into the 'Aggregated_Transactions' table
for index, row in df1.iterrows():
    insert_query= """insert into Aggregated_Transactions(
                        States,
                        Years,
                        Quarters,
                        Transaction_type,
                        Transaction_count,
                        Transaction_amount
                        )
                        values(%s,%s,%s,%s,%s,%s)"""
    

    value_query = (
        row["States"],
        row["Years"],
        row["Quarters"],
        row["Transaction_type"],
        row["Transaction_count"],
        row["Transaction_amount"]
    )
    cursor.execute(insert_query, value_query)
    mydb.commit()


In [8]:
import os
import json
import pandas as pd

# Aggregated_Users
Path2 = "D:/DS/Projects/project 2 - phonepe/pulse/data/aggregated/user/country/india/state/"
agg_users_list = os.listdir(Path2)

column2 = {"States": [], "Years": [], "Quarters": [], "Brands": [], "Transaction_count": [], "Percentage": []}


for state2 in agg_users_list:
    curr_states2 = Path2 + state2 + "/"
    agg_year_list2 = os.listdir(curr_states2)

    for years2 in agg_year_list2:
        curr_year2 = curr_states2 + years2 + "/"
        agg_file_list2 = os.listdir(curr_year2)

        for file2 in agg_file_list2:
            curr_file2 = curr_year2 + file2
            data2=open(curr_file2)
            
            B = json.load(data2)


            try:
                # Loop over usersByDevice to extract brand data
                for b in B["data"]["usersByDevice"]:
                    brand = b["brand"]
                    count = b["count"]
                    percentage = b["percentage"]

                    # Append data to respective lists
                    column2["Brands"].append(brand)
                    column2["Years"].append(years2)
                    column2["Quarters"].append(int(file2.strip(".json")))
                    column2["States"].append(state2)
                    column2["Transaction_count"].append(count)
                    column2["Percentage"].append(percentage)
            except:
                pass

df2 = pd.DataFrame(column2)
df2["States"] = df2["States"].replace(state_mapping)


In [9]:
df2

,States,Years,Quarters,Brands,Transaction_count,Percentage
0,Andaman & Nicobar,2018,1,Xiaomi,1665,0.247033
1,Andaman & Nicobar,2018,1,Samsung,1445,0.214392
2,Andaman & Nicobar,2018,1,Vivo,982,0.145697
3,Andaman & Nicobar,2018,1,Oppo,501,0.074332
4,Andaman & Nicobar,2018,1,OnePlus,332,0.049258
...,...,...,...,...,...,...
6727,West Bengal,2022,1,Lenovo,330017,0.015056
6728,West Bengal,2022,1,Infinix,284678,0.012987
6729,West Bengal,2022,1,Asus,280347,0.012790
6730,West Bengal,2022,1,Apple,277752,0.012671


In [10]:
# Connecting the PostgreSQL
mydb = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="Hannah$2686",
    database="phonepe",
    port="5432")

cursor = mydb.cursor()     # Creating a cursor to interact with the database

drop_query = """drop table if exists Aggregated_Users"""  # Drop the 'Aggregated_Users' table if it exists
cursor.execute(drop_query)
mydb.commit()


# Create the 'Aggregated_Users' table in PostgreSQL if it does not exist.
create_query = """
    create table if not exists Aggregated_Users(
    States varchar(100),
    Years int,
    Quarters int,
    Brands varchar(100),
    Transaction_count bigint,
    Percentage float
    )
    """
cursor.execute(create_query)
mydb.commit()

# Inserting information into the 'Aggregated_Users' table
for index, row in df2.iterrows():
    insert_query= """insert into Aggregated_Users(
                        States,
                        Years,
                        Quarters,
                        Brands,
                        Transaction_count,
                        Percentage
                        )
                        values(%s,%s,%s,%s,%s,%s)"""
    

    value_query = (
        row["States"],
        row["Years"],
        row["Quarters"],
        row["Brands"],
        row["Transaction_count"],
        row["Percentage"]
    )
    cursor.execute(insert_query, value_query)
    mydb.commit()




In [11]:
# Map Transactions

Path3="D:/DS/Projects/project 2 - phonepe/pulse/data/map/transaction/hover/country/india/state/"
map_trans_list=os.listdir(Path3)

column3={"States":[], "Years":[], "Quarters":[], "Districts":[], "Transaction_count":[], "Transaction_amount":[]}


for state3 in map_trans_list:
    curr_map1=Path3+state3+"/"
    map_year_list=os.listdir(curr_map1)

    for years3 in map_year_list:
        curr_year3=curr_map1+years3+"/"
        map_file_list1=os.listdir(curr_year3)
        
        for file3 in map_file_list1:
            curr_file3=curr_year3+file3
            data3=open(curr_file3)
            C=json.load(data3)

            for c in C["data"]["hoverDataList"]:
                name=c["name"]
                count=c["metric"][0]["count"]
                amount=c["metric"][0]["amount"]

                column3["Districts"].append(name)
                column3["Years"].append(years3)
                column3["Quarters"].append(int(file3.strip(".json")))
                column3["States"].append(state3)
                column3["Transaction_count"].append(count)
                column3["Transaction_amount"].append(amount)

df3=pd.DataFrame(column3)
df3["States"] = df3["States"].replace(state_mapping)


In [12]:
df3

,States,Years,Quarters,Districts,Transaction_count,Transaction_amount
0,Andaman & Nicobar,2018,1,north and middle andaman district,442,9.316631e+05
1,Andaman & Nicobar,2018,1,south andaman district,5688,1.256025e+07
2,Andaman & Nicobar,2018,1,nicobars district,528,1.139849e+06
3,Andaman & Nicobar,2018,2,north and middle andaman district,825,1.317863e+06
4,Andaman & Nicobar,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
16827,West Bengal,2023,3,nadia district,30146530,5.787869e+10
16828,West Bengal,2023,3,birbhum district,17597047,3.090737e+10
16829,West Bengal,2023,3,purba medinipur district,34244893,6.813718e+10
16830,West Bengal,2023,3,maldah district,28539407,5.299766e+10


In [13]:
# Connecting the PostgreSQL
mydb = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="Hannah$2686",
    database="phonepe",
    port="5432")

cursor = mydb.cursor()     # Creating a cursor to interact with the database

drop_query = """drop table if exists Mapped_Transactions"""  # Drop the 'Mapped_Transactions' table if it exists
cursor.execute(drop_query)
mydb.commit()

# Create the 'Mapped_Transactions' table in PostgreSQL if it does not exist.
create_query = """
    create table if not exists Mapped_Transactions(
    States varchar(100),
    Years int,
    Quarters int,
    Districts varchar(100),
    Transaction_count bigint,
    Transaction_amount float
    )
    """
cursor.execute(create_query)
mydb.commit()

# Inserting information into the 'Mapped_Transactions' table
for index, row in df3.iterrows():
    insert_query= """insert into Mapped_Transactions(
                        States,
                        Years,
                        Quarters,
                        Districts,
                        Transaction_count,
                        Transaction_amount
                        )
                        values(%s,%s,%s,%s,%s,%s)"""
    

    value_query = (
        row["States"],
        row["Years"],
        row["Quarters"],
        row["Districts"],
        row["Transaction_count"],
        row["Transaction_amount"]
    )
    cursor.execute(insert_query, value_query)
    mydb.commit()


In [14]:
# Map Users

Path4="D:/DS/Projects/project 2 - phonepe/pulse/data/map/user/hover/country/india/state/"
map_user_list=os.listdir(Path4)

column4={"States":[], "Years":[], "Quarters":[], "Districts":[], "Registered_user":[], "App_opens":[]}


for state4 in map_user_list:
    curr_map2=Path4+state4+"/"
    map_year_list1=os.listdir(curr_map2)

    for years4 in map_year_list1:
        curr_year4=curr_map2+years4+"/"
        map_file_list2=os.listdir(curr_year4)
        
        for file4 in map_file_list2:
            curr_file4=curr_year4+file4
            data4=open(curr_file4)
            D=json.load(data4)
            
            for d in D["data"]["hoverData"].items():
                district=d[0]
                register_user=d[1]["registeredUsers"]
                app_opens=d[1]["appOpens"]

                column4["Districts"].append(district)
                column4["Years"].append(years4)
                column4["Quarters"].append(int(file4.strip(".json")))
                column4["States"].append(state4)
                column4["Registered_user"].append(register_user)
                column4["App_opens"].append(app_opens)

df4=pd.DataFrame(column4)
df4["States"] = df4["States"].replace(state_mapping)

In [15]:
df4

,States,Years,Quarters,Districts,Registered_user,App_opens
0,Andaman & Nicobar,2018,1,north and middle andaman district,632,0
1,Andaman & Nicobar,2018,1,south andaman district,5846,0
2,Andaman & Nicobar,2018,1,nicobars district,262,0
3,Andaman & Nicobar,2018,2,north and middle andaman district,911,0
4,Andaman & Nicobar,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
16831,West Bengal,2023,3,nadia district,1544294,12674328
16832,West Bengal,2023,3,birbhum district,974363,18961683
16833,West Bengal,2023,3,purba medinipur district,1530289,18593769
16834,West Bengal,2023,3,maldah district,1091634,28020285


In [16]:
# Connecting the PostgreSQL
mydb = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="Hannah$2686",
    database="phonepe",
    port="5432")

cursor = mydb.cursor()     # Creating a cursor to interact with the database

drop_query = """drop table if exists Mapped_Users"""  # Drop the 'Mapped_Users' table if it exists
cursor.execute(drop_query)
mydb.commit()

# Create the 'Mapped_Users' table in PostgreSQL if it does not exist.
create_query = """
    create table if not exists Mapped_Users(
    States varchar(100),
    Years int,
    Quarters int,
    Districts varchar(100),
    Registered_user bigint,
    App_opens int
    )
    """
cursor.execute(create_query)
mydb.commit()

# Inserting information into the 'mapped users' table
for index, row in df4.iterrows():
    insert_query= """insert into Mapped_Users(
                        States,
                        Years,
                        Quarters,
                        Districts,
                        Registered_user,
                        App_opens
                        )
                        values(%s,%s,%s,%s,%s,%s)"""
    

    value_query = (
        row["States"],
        row["Years"],
        row["Quarters"],
        row["Districts"],
        row["Registered_user"],
        row["App_opens"]
    )
    cursor.execute(insert_query, value_query)
    mydb.commit()


In [17]:
# Top Transactions District

Path5="D:/DS/Projects/project 2 - phonepe/pulse/data/top/transaction/country/india/state/"
top_trans_list=os.listdir(Path5)

column5={"States":[], "Years":[], "Quarters":[], "Districts":[], "Transaction_count":[], "Transaction_amount":[]}

for state5 in top_trans_list:
    curr_top1=Path5+state5+"/"
    top_year_list=os.listdir(curr_top1)

    for years5 in top_year_list:
        curr_year5=curr_top1+years5+"/"
        top_file_list1=os.listdir(curr_year5)
        
        for file5 in top_file_list1:
            top_file1=curr_year5+file5
            data5=open(top_file1)

            E=json.load(data5)

            for e in E["data"]["districts"]:
                district=e["entityName"]
                count=e["metric"]["count"]
                amount=e["metric"]["amount"]

                column5["Districts"].append(district)
                column5["Years"].append(years5)
                column5["Quarters"].append(int(file5.strip(".json")))
                column5["States"].append(state5)
                column5["Transaction_count"].append(count)
                column5["Transaction_amount"].append(amount)

df5=pd.DataFrame(column5)
df5["States"] = df5["States"].replace(state_mapping)

In [18]:
df5

,States,Years,Quarters,Districts,Transaction_count,Transaction_amount
0,Andaman & Nicobar,2018,1,south andaman,5688,1.256025e+07
1,Andaman & Nicobar,2018,1,nicobars,528,1.139849e+06
2,Andaman & Nicobar,2018,1,north and middle andaman,442,9.316631e+05
3,Andaman & Nicobar,2018,2,south andaman,9395,2.394824e+07
4,Andaman & Nicobar,2018,2,nicobars,1120,3.072437e+06
...,...,...,...,...,...,...
6803,West Bengal,2023,3,hooghly,32021392,5.624223e+10
6804,West Bengal,2023,3,howrah,31734226,5.800237e+10
6805,West Bengal,2023,3,paschim bardhaman,30585672,4.340349e+10
6806,West Bengal,2023,3,paschim medinipur,30418455,5.043177e+10


In [19]:
# Connecting the PostgreSQL
mydb = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="Hannah$2686",
    database="phonepe",
    port="5432")

cursor = mydb.cursor()     # Creating a cursor to interact with the database

drop_query = """drop table if exists top_transactions_districts"""  # Drop the 'top_transactions_districts' table if it exists
cursor.execute(drop_query)
mydb.commit()

# Create the 'top_transactions_districts' table in PostgreSQL if it does not exist.
create_query = """
    create table if not exists top_transactions_districts(
    States varchar(100),
    Years int,
    Quarters int,
    Districts varchar(100),
    Transaction_count bigint,
    Transaction_amount bigint
    )
    """
cursor.execute(create_query)
mydb.commit()

# Inserting information into the 'top_transactions_districts' table
for index, row in df5.iterrows():
    insert_query= """insert into top_transactions_districts(
                        States,
                        Years,
                        Quarters,
                        Districts,
                        Transaction_count,
                        Transaction_amount
                        )
                        values(%s,%s,%s,%s,%s,%s)"""
    

    value_query = (
        row["States"],
        row["Years"],
        row["Quarters"],
        row["Districts"],
        row["Transaction_count"],
        row["Transaction_amount"]
    )
    cursor.execute(insert_query, value_query)
    mydb.commit()


In [20]:
# Top User District

Path6="D:/DS/Projects/project 2 - phonepe/pulse/data/top/user/country/india/state/"
top_user_list=os.listdir(Path6)

column6={"States":[], "Years":[], "Quarters":[], "Districts":[], "Registered_user":[]}

for state6 in top_user_list:
    curr_top2=Path6+state6+"/"
    top_year_list1=os.listdir(curr_top2)

    for years6 in top_year_list1:
        curr_year6=curr_top2+years6+"/"
        top_file_list2=os.listdir(curr_year6)
        
        for file6 in top_file_list2:
            top_file2=curr_year6+file6
            data6=open(top_file2)

            F=json.load(data6)

            for f in F["data"]["districts"]:
                district=f["name"]
                register_user1=f["registeredUsers"]

                column6["Districts"].append(district)
                column6["Years"].append(years6)
                column6["Quarters"].append(int(file6.strip(".json")))
                column6["States"].append(state6)
                column6["Registered_user"].append(register_user1)

df6=pd.DataFrame(column6)
df6["States"] = df6["States"].replace(state_mapping)

In [21]:
df6

,States,Years,Quarters,Districts,Registered_user
0,Andaman & Nicobar,2018,1,south andaman,5846
1,Andaman & Nicobar,2018,1,north and middle andaman,632
2,Andaman & Nicobar,2018,1,nicobars,262
3,Andaman & Nicobar,2018,2,south andaman,8143
4,Andaman & Nicobar,2018,2,north and middle andaman,911
...,...,...,...,...,...
6803,West Bengal,2023,3,howrah,1608895
6804,West Bengal,2023,3,nadia,1544294
6805,West Bengal,2023,3,purba medinipur,1530289
6806,West Bengal,2023,3,paschim medinipur,1384076


In [22]:
# Connecting the PostgreSQL
mydb = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="Hannah$2686",
    database="phonepe",
    port="5432")

cursor = mydb.cursor()     # Creating a cursor to interact with the database

drop_query = """drop table if exists Top_users_districts"""  # Drop the 'Top_users_districts' table if it exists
cursor.execute(drop_query)
mydb.commit()

# Create the 'Top_users_districts' table in PostgreSQL if it does not exist.
create_query = """
    create table if not exists Top_users_districts(
    States varchar(100),
    Years int,
    Quarters int,
    Districts varchar(100),
    Registered_user bigint
    )
    """
cursor.execute(create_query)
mydb.commit()

# Inserting  information into the 'Top_users_districts' table
for index, row in df6.iterrows():
    insert_query= """insert into Top_users_districts(
                        States,
                        Years,
                        Quarters,
                        Districts,
                        Registered_user
                        )
                        values(%s,%s,%s,%s,%s)"""
    

    value_query = (
        row["States"],
        row["Years"],
        row["Quarters"],
        row["Districts"],
        row["Registered_user"]
    )
    cursor.execute(insert_query, value_query)
    mydb.commit()


In [23]:
# Top Transactions Pincode

Path7="D:/DS/Projects/project 2 - phonepe/pulse/data/top/transaction/country/india/state/"
top_trans_pin_list=os.listdir(Path7)

column7={"States":[], "Years":[], "Quarters":[], "Pincode":[], "Transaction_count":[], "Transaction_amount":[]}

for state7 in top_trans_pin_list:
    curr_top_pin1=Path7+state7+"/"
    top_pin_year_list=os.listdir(curr_top_pin1)

    for years7 in top_pin_year_list:
        curr_year7=curr_top_pin1+years7+"/"
        top_pin_file_list1=os.listdir(curr_year7)
        
        for file7 in top_pin_file_list1:
            top_pin_file1=curr_year7+file7
            data7=open(top_pin_file1)

            G=json.load(data7)

            for g in G["data"]["pincodes"]:
                pin=g["entityName"]
                count = g["metric"]["count"]
                amount=g["metric"]["amount"]

                column7["Pincode"].append(pin)
                column7["Years"].append(years7)
                column7["Quarters"].append(int(file7.strip(".json")))
                column7["States"].append(state7)
                column7["Transaction_count"].append(count)
                column7["Transaction_amount"].append(amount)

df7=pd.DataFrame(column7)
df7["States"] = df7["States"].replace(state_mapping)


In [24]:
df7

,States,Years,Quarters,Pincode,Transaction_count,Transaction_amount
0,Andaman & Nicobar,2018,1,744101,1622,2.769298e+06
1,Andaman & Nicobar,2018,1,744103,1223,2.238042e+06
2,Andaman & Nicobar,2018,1,744102,969,3.519060e+06
3,Andaman & Nicobar,2018,1,744105,685,1.298561e+06
4,Andaman & Nicobar,2018,1,744104,340,1.039715e+06
...,...,...,...,...,...,...
8205,West Bengal,2023,3,700001,3536928,7.193522e+09
8206,West Bengal,2023,3,721301,3464729,4.871566e+09
8207,West Bengal,2023,3,700039,3405551,5.095825e+09
8208,West Bengal,2023,3,732125,3347165,5.353831e+09


In [25]:
# Connecting the PostgreSQL
mydb = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="Hannah$2686",
    database="phonepe",
    port="5432")

cursor = mydb.cursor()     # Creating a cursor to interact with the database

drop_query = """drop table if exists top_transactions_Pincode"""  # Drop the 'top_transactions_Pincode' table if it exists
cursor.execute(drop_query)
mydb.commit()

# Create the 'top_transactions_Pincode' table in PostgreSQL if it does not exist.
create_query = """
    create table if not exists top_transactions_Pincode(
    States varchar(100),
    Years int,
    Quarters int,
    Pincode varchar(100),
    Transaction_count bigint,
    Transaction_amount float 
    )
    """
cursor.execute(create_query)
mydb.commit()

# Inserting  information into the 'top_transactions_Pincode' table
for index, row in df7.iterrows():
    insert_query= """insert into top_transactions_Pincode(
                        States,
                        Years,
                        Quarters,
                        Pincode,
                        Transaction_count,
                        Transaction_amount
                        )
                        values(%s,%s,%s,%s,%s,%s)"""
    

    value_query = (
        row["States"],
        row["Years"],
        row["Quarters"],
        row["Pincode"],
        row["Transaction_count"],
        row["Transaction_amount"]
    )
    cursor.execute(insert_query, value_query)
    mydb.commit()


In [26]:
# Top User Pincode

Path8="D:/DS/Projects/project 2 - phonepe/pulse/data/top/user/country/india/state/"
top_tran_pin_list1=os.listdir(Path8)

column8={"States":[], "Years":[], "Quarters":[], "Pincodes":[], "Registered_user":[]}

for state8 in top_tran_pin_list1:
    curr_top_pin2=Path8+state8+"/"
    top_pin_year_list2=os.listdir(curr_top_pin2)

    for years8 in top_pin_year_list2:
        curr_year8=curr_top_pin2+years8+"/"
        top_pin_file_list2=os.listdir(curr_year8)
        
        for file8 in top_pin_file_list2:
            top_pin_file2=curr_year8+file8
            data8=open(top_pin_file2)

            H=json.load(data8)
            
            for h in H["data"]["pincodes"]:
                pincode=h["name"]
                register_user2=h["registeredUsers"]

                column8["Pincodes"].append(pincode)
                column8["Years"].append(years8)
                column8["Quarters"].append(int(file8.strip(".json")))
                column8["States"].append(state8)
                column8["Registered_user"].append(register_user2)

df8=pd.DataFrame(column8)
df8["States"] = df8["States"].replace(state_mapping)

In [27]:
df8

,States,Years,Quarters,Pincodes,Registered_user
0,Andaman & Nicobar,2018,1,744103,1608
1,Andaman & Nicobar,2018,1,744101,1108
2,Andaman & Nicobar,2018,1,744105,1075
3,Andaman & Nicobar,2018,1,744102,1006
4,Andaman & Nicobar,2018,1,744104,272
...,...,...,...,...,...
8206,West Bengal,2023,3,700015,122316
8207,West Bengal,2023,3,742304,119245
8208,West Bengal,2023,3,721101,118798
8209,West Bengal,2023,3,700150,114987


In [28]:
# Connecting the PostgreSQL
mydb = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="Hannah$2686",
    database="phonepe",
    port="5432")

cursor = mydb.cursor()     # Creating a cursor to interact with the database

drop_query = """drop table if exists Top_users_pincode"""  # Drop the 'Top_users_pincode' table if it exists
cursor.execute(drop_query)
mydb.commit()

# Create the 'Top_users_pincode' table in PostgreSQL if it does not exist.
create_query = """
    create table if not exists Top_users_pincode(
    States varchar(100),
    Years int,
    Quarters int,
    Pincodes text,
    Registered_user int
    
    )
    """
cursor.execute(create_query)
mydb.commit()

# Inserting  information into the 'Top_users_pincode' table
for index, row in df8.iterrows():
    insert_query= """insert into Top_users_pincode(
                        States,
                        Years,
                        Quarters,
                        Pincodes,
                        Registered_user
                        )
                        values(%s,%s,%s,%s,%s)"""
    

    value_query = (
        row["States"],
        row["Years"],
        row["Quarters"],
        row["Pincodes"],
        row["Registered_user"]
    )
    cursor.execute(insert_query, value_query)
    mydb.commit()


In [29]:
import os
import json
import pandas as pd
import psycopg2
import streamlit as st
import plotly.express as px
import numpy as np
import requests

# Connecting the PostgreSQL
mydb = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="Hannah$2686",
    database="phonepe",
    port="5432"
)

cursor = mydb.cursor()

# streamlit creation

# Setting page configuration
st.set_page_config(page_title="Phonepe Pulse Data Visualization", layout="wide", page_icon="point_right")
st.header("Phonepe Pulse Data Visualization and Exploration", divider='rainbow')
st.sidebar.header(":wave: :violet[Hello! Welcome to the dashboard]")

col1, col2 = st.columns(2)

with col1:
    selected_quarters = st.multiselect(
        'Please select a Quarter', ('1', '2', '3', '4'))
    
with col2:
    selected_years = st.multiselect(
        'Please select a Year', (2018, 2019, 2020, 2021, 2022, 2023))

quarters_filter = ", ".join(selected_quarters)
years_filter = ", ".join(map(str, selected_years))

2024-03-16 10:50:00.984 
  command:

    streamlit run d:\DS\Projects\project 1 - Youtube\.venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [30]:
selected_quarters

[]

In [31]:
import os
import json
import pandas as pd
import psycopg2
import streamlit as st
import plotly.express as px
import numpy as np
import requests

# Connecting the PostgreSQL
mydb = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="Hannah$2686",
    database="phonepe",
    port="5432"
)

cursor = mydb.cursor()

# streamlit creation

# Setting page configuration
st.set_page_config(page_title="Phonepe Pulse Data Visualization", layout="wide", page_icon="point_right")
st.header("Phonepe Pulse Data Visualization and Exploration", divider='rainbow')
st.sidebar.header(":wave: :violet[Hello! Welcome to the dashboard]")

DeltaGenerator(_root_container=1, _parent=DeltaGenerator())

In [32]:
def agg_trans():
    col1, col2 = st.columns(2)

    with col1:
        selected_quarters1 = st.selectbox(
            'Please select a Quarter', ('1', '2', '3', '4'))
        if not selected_quarters1:
            st.warning("Please select at least one Quarter.")
        
    with col2:
        selected_years1 = st.selectbox(
            'Please select a Year', (2018, 2019, 2020, 2021, 2022, 2023))
        if not selected_years1:
            st.warning("Please select at least one Year.")
        
    # Constructing the SQL query based on selected quarters and years
    if selected_quarters1 and selected_years1:
        quarters_filter1 = selected_quarters1
        years_filter1 = selected_years1

        query = f"""SELECT states, years, quarters, SUM(transaction_count) / 100000 AS trxn_CNT_in_lakhs, SUM(transaction_amount) / 100000 AS trxn_AMT_in_lakhs
                    FROM aggregated_transactions
                    WHERE quarters IN ({quarters_filter1}) AND years IN ({years_filter1})
                    GROUP BY states, years, quarters
                    ORDER BY states"""

        cursor.execute(query)  # Execute the query
        mydb.commit()
        t1 = cursor.fetchall()        
        df1 = pd.DataFrame(t1, columns=["states", "years", "quarters", "trxn_CNT_in_lakhs", "trxn_AMT_in_lakhs"])
        fig = px.choropleth(
            df1,
            geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
            featureidkey='properties.ST_NM',
            locations='states',
            color='states',
            hover_data=['states', "years", "quarters", 'trxn_CNT_in_lakhs', "trxn_AMT_in_lakhs"],
            title="Aggregated_transaction",
            color_continuous_scale='blues'
        )

        fig.update_geos(fitbounds="locations", visible=False)
        st.plotly_chart(fig)

In [33]:
def agg_users():
    col1, col2 = st.columns(2)

    with col1:
        selected_quarters2 = st.selectbox('Please select a Quarter', ('4', '3', '2', '1'))
        
    with col2:
        selected_years2 = st.selectbox(
            'Please select a Year', (2023, 2022, 2021, 2020, 2019, 2018))
        
    # Constructing the SQL query based on selected quarters and years
    if selected_quarters2 and selected_years2:
        quarters_filter2 = selected_quarters2
        years_filter2 = selected_years2

        query = f"""select states, years, quarters, brands, SUM(transaction_count), registered_users
                    from aggregated_users
                    WHERE quarters IN ({quarters_filter2}) AND years IN ({years_filter2})
                    group by states, years, quarters, brands, registered_users"""

        cursor.execute(query)  # Execute the query
        mydb.commit()
        t2 = cursor.fetchall()        
        df2 = pd.DataFrame(t2, columns=["states", "years", "quarters", "brands", "transaction_count","registered_users"])
        fig1 = px.choropleth(
            df2,
            geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
            featureidkey='properties.ST_NM',
            locations='states',
            color='states',
            hover_data=["states", "years", "quarters", "brands","transaction_count","registered_users"],
            title="Aggregated_Users",
            color_continuous_scale='blues'
        )

        fig1.update_geos(fitbounds="locations", visible=False)
        st.plotly_chart(fig1)